# Portfolio Optimization with the Markowitz Model

This notebook demonstrates how to use real historical stock data to build an optimal portfolio using the Markowitz mean-variance optimization approach.

In [ ]:
# Install dependencies (uncomment if running in Colab or a fresh environment)
# !pip install numpy pandas matplotlib yfinance scipy

## 1. Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.optimize import minimize

## 2. Download Stock Data

In [ ]:
# Choose your stock tickers (edit this list as you like)
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META']
data = yf.download(tickers, start="2020-01-01", end="2024-12-31")['Adj Close']
data = data.dropna()
data.tail()

## 3. Calculate Returns

In [ ]:
returns = data.pct_change().dropna()
mean_returns = returns.mean() * 252  # annualized
cov_matrix = returns.cov() * 252     # annualized
returns.tail()

## 4. Simulate Random Portfolios (Monte Carlo)

In [ ]:
num_portfolios = 10000
results = np.zeros((3 + len(tickers), num_portfolios))

for i in range(num_portfolios):
    weights = np.random.random(len(tickers))
    weights /= np.sum(weights)
    portfolio_return = np.dot(weights, mean_returns)
    portfolio_stddev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe = portfolio_return / portfolio_stddev
    results[0, i] = portfolio_return
    results[1, i] = portfolio_stddev
    results[2, i] = sharpe
    for j in range(len(weights)):
        results[j+3, i] = weights[j]

## 5. Plot Efficient Frontier

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(results[1,:], results[0,:], c=results[2,:], cmap='viridis', marker='o', s=10, alpha=0.3)
plt.xlabel('Volatility (Std. Deviation)')
plt.ylabel('Return')
plt.colorbar(label='Sharpe Ratio')
plt.title('Portfolio Optimization - Markowitz Efficient Frontier')
plt.show()

## 6. Find the Optimal Portfolio (Max Sharpe Ratio)

In [ ]:
def neg_sharpe(weights, mean_returns, cov_matrix):
    portfolio_return = np.dot(weights, mean_returns)
    portfolio_stddev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return -portfolio_return / portfolio_stddev

constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bounds = tuple((0, 1) for _ in range(len(tickers)))
initial_guess = len(tickers) * [1. / len(tickers)]

optimal = minimize(neg_sharpe, initial_guess, args=(mean_returns, cov_matrix), method='SLSQP', bounds=bounds, constraints=constraints)
optimal_weights = optimal.x
print('Optimal weights:')
for ticker, weight in zip(tickers, optimal_weights):
    print(f'{ticker}: {weight:.2%}')

## 7. Show Results in a Table

In [ ]:
result_df = pd.DataFrame({'Ticker': tickers, 'Optimal Weight': optimal_weights})
result_df['Optimal Weight'] = result_df['Optimal Weight'].map(lambda x: f'{x:.2%}')
result_df

---
You can now edit the tickers or time range and explore further!

**Author:** Majed Abdulaziz

**Date:** August 2025